In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
sentences = [
    '<br>Today is a rainy day',
    'Today is a sunny day',
    'Is it sunny Today?',
    'I really enjoyed walking in the snow today'
]

In [3]:
# num_words here represent the number of tokens (number of distinct words)
tokenizer = Tokenizer(num_words = 100, oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'day': 5, 'sunny': 6, 'br': 7, 'rainy': 8, 'it': 9, 'i': 10, 'really': 11, 'enjoyed': 12, 'walking': 13, 'in': 14, 'the': 15, 'snow': 16}


In [4]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[7, 2, 3, 4, 8, 5], [2, 3, 4, 6, 5], [3, 9, 6, 2], [10, 11, 12, 13, 14, 15, 16, 2]]


In [5]:
test_sentences = [
    'Today is a snowy day',
    'Will it be rainy tomorrow?'
]
test_sequences = tokenizer.texts_to_sequences(test_sentences)
print(test_sequences)

[[2, 3, 4, 1, 5], [1, 9, 1, 8, 1]]


In [6]:
# Default is having tokens prepadded
padded = pad_sequences(sequences)
print(padded)

[[ 0  0  7  2  3  4  8  5]
 [ 0  0  0  2  3  4  6  5]
 [ 0  0  0  0  3  9  6  2]
 [10 11 12 13 14 15 16  2]]


In [7]:
# Trying out post padded
padded_post = pad_sequences(sequences, padding='post')
print(padded_post)

[[ 7  2  3  4  8  5  0  0]
 [ 2  3  4  6  5  0  0  0]
 [ 3  9  6  2  0  0  0  0]
 [10 11 12 13 14 15 16  2]]


In [8]:
# Trying out max length (truncating from the front)
max_padded = pad_sequences(sequences, padding='post', maxlen=6)
print(max_padded)

[[ 7  2  3  4  8  5]
 [ 2  3  4  6  5  0]
 [ 3  9  6  2  0  0]
 [12 13 14 15 16  2]]


In [9]:
# Trying out max length (truncating from behind)
max_padded_post = pad_sequences(sequences, padding='post', maxlen=6, truncating='post')
print(max_padded_post)

[[ 7  2  3  4  8  5]
 [ 2  3  4  6  5  0]
 [ 3  9  6  2  0  0]
 [10 11 12 13 14 15]]


In [10]:
# Adapting BeautifulSoup to remove stopwords
from bs4 import BeautifulSoup
import string

stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]
table = str.maketrans('', '', string.punctuation)

In [11]:
# Using TFDS Dataset
import tensorflow_datasets as tfds
imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split = 'train'))
for item in train_data:
    sentence = str(item['text'].decode('UTF-8').lower())
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    
    # Strips these punctuations in between words so words can be again tokenized based on single words, for e.g. 'him/her' will 
    # be split into 'him / her' then '/'' will be removed and him and her can be tokenized rather than having himher as a token
    sentence = sentence.replace('/', ' / ')
    sentence = sentence.replace('-', ' - ')
    sentence = sentence.replace(',', ' , ')
    sentence = sentence.replace('.', ' . ')
    words = sentence.split()
    filtered_sentence = ''
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    imdb_sentences.append(filtered_sentence)

C:\Users\liewy\AppData\Local\Temp\ipykernel_23528\2732870245.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


In [12]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 55000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)

In [13]:
print(tokenizer.word_index)

{'movie': 1, 'film': 2, 'good': 3, 'time': 4, 'story': 5, 'bad': 6, 'people': 7, 'great': 8, 'dont': 9, 'movies': 10, 'films': 11, 'characters': 12, 'watch': 13, 'character': 14, 'life': 15, 'plot': 16, 'acting': 17, 'love': 18, 'man': 19, 'scene': 20, 'scenes': 21, 'real': 22, 'watching': 23, 'years': 24, 'doesnt': 25, 'actors': 26, 'thing': 27, 'didnt': 28, 'work': 29, 'funny': 30, '10': 31, 'director': 32, 'lot': 33, 'cast': 34, 'things': 35, 'pretty': 36, 'young': 37, 'horror': 38, 'fact': 39, 'big': 40, 'long': 41, 'thought': 42, 'series': 43, 'original': 44, 'ive': 45, 'action': 46, 'times': 47, 'comedy': 48, 'point': 49, 'isnt': 50, 'role': 51, 'interesting': 52, 'family': 53, 'bit': 54, 'music': 55, 'script': 56, 'making': 57, 'guy': 58, 'feel': 59, 'minutes': 60, 'performance': 61, '2': 62, 'kind': 63, 'worst': 64, 'girl': 65, 'day': 66, 'fun': 67, 'hard': 68, 'woman': 69, 'played': 70, 'screen': 71, 'set': 72, 'place': 73, 'book': 74, 'money': 75, 'true': 76, 'sense': 77, 're

In [14]:
# Testing out tokenizer with stopwords removed
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[320, 4758, 66], [320, 5994, 66], [4758, 320]]


In [15]:
# Get words using sequences
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
for sequence in sequences:
    decoded_review = ' '.join([reverse_word_index.get(i, '?') for i in sequence])
    print(decoded_review)

today sunny day
today rainy day
sunny today


In [16]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised = True,
    with_info=True
)

In [17]:
encoder = info.features['text'].encoder
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [18]:
print(encoder.subwords)

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_', 't_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', ' (', 'are_', 'his_', 'have_', 'film_', 'not_', 'ing_', 'be_', 'ed_', 'you_', ' "', 'it', 'd_', 'an_', 'he_', 'by_', 'at_', 'one_', 'who_', 'y_', 'from_', 'e_', 'or_', 'all_', 'like_', 'they_', '" ', 'so_', 'just_', 'has_', ') ', 'her_', 'about_', 'out_', 'This_', 'some_', 'ly_', 'movie', 'film', 'very_', 'more_', 'It_', 'would_', 'what_', 'when_', 'which_', 'good_', 'if_', 'up_', 'only_', 'even_', 'their_', 'had_', 'really_', 'my_', 'can_', 'no_', 'were_', 'see_', 'she_', '? ', 'than_', '! ', 'there_', 'get_', 'been_', 'into_', ' - ', 'will_', 'much_', 'story_', 'because_', 'ing', 'time_', 'n_', 'we_', 'ed', 'me_', ': ', 'most_', 'other_', 'don', 'do_', 'm_', 'es_', 'how_', 'also_', 'make_', 'its_', 'could_', 'first_', 'any_', "' ", 'people_', 'great_', 've_', 'ly', 'er_', 'made_', 'r_', 'B

In [19]:
sample_string = 'Today is a sunny day'
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

Encoded string is [6427, 4869, 9, 4, 2365, 1361, 606]


In [20]:
print(encoder.subwords[6426])

Tod


In [21]:
encoded_string = encoder.encode(sample_string)
original_string = encoder.decode(encoded_string)
test_string = encoder.decode([6427, 4869, 9, 4, 2365, 1361, 606])

In [22]:
print(test_string)

Today is a sunny day


In [34]:
# Reading JSON Files
import json

sentences = []
labels = []
urls = []

with open("Sarcasm_Headlines_Dataset.json", 'r') as f:
    for line in f:
        item = json.loads(line)
        sentence = item['headline'].lower()
        sentence = sentence.replace(",", " , ")
        sentence = sentence.replace(".", " . ")
        sentence = sentence.replace("-", " - ")
        sentence = sentence.replace("/", " / ")
        soup = BeautifulSoup(sentence)
        wourds = sentence.split()
        filtered_sentence = ""
        for word in words:
            word = word.translate(table)
            if word not in stopwords:
                filtered_sentence = filtered_sentence + word + " "
        sentences.append(filtered_sentence)
        labels.append(item['is_sarcastic'])
        urls.append(item['article_link'])

C:\Users\liewy\AppData\Local\Temp\ipykernel_23528\4290755776.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(sentence)


In [37]:
training_size = 23000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0: training_size]
testing_labels = labels[training_size:]

In [38]:
vocab_size =20000
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(training_sequences, padding = 'post')
print(word_index)

{'<OOV>': 1, 'riget': 2, 'animation': 3, 'story': 4, 'actors': 5, 'good': 6, 'great': 7, 'helmer': 8, 'lars': 9, 'bit': 10, 'experiment': 11, 'create': 12, 'puppet': 13, '3': 14, 'wonderful': 15, 'comedy': 16, 'mystery': 17, 'thriller': 18, 'time': 19, 'nice': 20, 'combination': 21, 'strange': 22, 'dogma': 23, 'style': 24, 'telling': 25, 'music': 26, 'unacceptable': 27, 'thinking': 28, 'continue': 29, 'drusse': 30, 'idea': 31, 'iii': 32, '3rd': 33, 'suggestion': 34, 'mayble': 35, 'stupid': 36, 'likes': 37, 'small': 38, 'riget3': 39, 'solution': 40, 'driven': 41, 'team': 42, 'america': 43, 'trey': 44, 'parker': 45, 'computer': 46, 'real': 47, 'principle': 48, 'work': 49, 'alive': 50, 'fans': 51, 'shouldnt': 52, 'big': 53, 'difference': 54, 'average': 55, 'watcher': 56, 'normal': 57, 'shots': 58, 'movie': 59, 'thing': 60, 'completely': 61, 'understandable': 62, 'nowadays': 63, 'play': 64, 'possibilities': 65, 'marketing': 66, 'director': 67, 'finishes': 68, 'trilogy': 69, '10': 70, 'year

In [ ]:
# Filtering off unwanted word by tokenization is the key in this chapter. This is by far the most important step to understanding natural language processing